In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import collections
from typing import Sequence, Tuple, List, Optional
from tracker import extractor, utils, metrics, visuals

In [ ]:
def cartesian(position: np.ndarray, order: List[str]) -> np.ndarray:
    phi, r, z = [position[order.index(s)] for s in ["phi", "r", "z"]]
    return np.array([(np.cos(phi) * r), (np.sin(phi) * r), z])

def groupby(array: np.ndarray) -> List[List[float]]:
    D = dict((value, index) for (index, value) in enumerate(np.unique(array)))
    L = [[] for _ in range(len(D))]
    for index, value in enumerate(array):
        L[D[value]].append(index)
    return L

In [120]:
def circle_circle_intersection(
        p0 : np.ndarray,  # The center of the first circle.
        r0 : float,       # The radius of the first circle.
        p1 : np.ndarray,  # The center of the second circle.
        r1 : float,       # The radius of the second circle.
        ) -> Tuple[np.ndarray, np.ndarray]:  # Two points of intersection.
    d  = np.linalg.norm(p0 - p1)
    a  = (r0 * r0 - r1 * r1 + d * d) / (2 * d)
    h  = np.sqrt(r0 * r0 - a * a)
    p2 = (p1 - p0) * (a / d) + p0
    x3 = p2[0] + h * (p1[1] - p0[1]) / d
    y3 = p2[1] - h * (p1[0] - p0[0]) / d
    x4 = p2[0] - h * (p1[1] - p0[1]) / d
    y4 = p2[1] + h * (p1[0] - p0[0]) / d
    return np.array([x3, y3]), np.array([x4, y4])

def circle_line_intersection(
        p0 : np.ndarray,  # The position (xy) of the ray's starting point.
        p1 : np.ndarray,  # The position (xy) of the ray's guiding point.
        p2 : np.ndarray,  # The position (xy) of the center of the circle.
        r  : float,       # The radius of the circle.
        ) -> Optional[np.ndarray]:  # The first point of intersection or None if no intersection.
    v = p1 - p0
    a = v.dot(v)
    b = 2 * v.dot(p0 - p2)
    c = p0.dot(p0) + p2.dot(p2) - 2 * p0.dot(p2) - r**2
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:  # Test for negative discriminant.
        return None  # If discriminant is negative, then no solution.
    t = (-b + np.sqrt(discriminant)) / (2 * a)
    return p0 + t * v

def tighten_bounds(bounds1: Tuple[float, float], bounds2: Tuple[float, float]) -> Tuple[float, float]:
    if bounds1 == (0, 2 * np.pi):
        return bounds2
    if bounds2 == (0, 2 * np.pi):
        return bounds1
    b1 = bounds1[0] if phi_lies_within_bounds(bounds2[0], bounds1) else bounds2[0]
    b2 = bounds1[1] if phi_lies_within_bounds(bounds2[1], bounds1) else bounds2[1]
    return b1, b2

def phi_lies_within_bounds(phi: float, bounds: Tuple[float, float]) -> bool:
    phi = phi % (2 * np.pi)
    b1  = bounds[0] % (2 * np.pi)
    b2  = bounds[1] % (2 * np.pi)
    return (b1 <= phi or phi <= b2) if (b1 > b2) else (b1 <= phi and phi <= b2)

def phi_bounds_from_circle(xyz: np.ndarray, radius: float) -> Tuple[float, float]:
    center = xyz[:2] / 2
    b0, b1 = circle_circle_intersection(center, np.linalg.norm(center), np.array([0, 0]), radius)
    return  np.arctan2(b1[1], b1[0]), np.arctan2(b0[1], b0[0])

def phi_bounds_from_lines(super_xyz: np.ndarray, sub_xyz: np.ndarray, radius: float) -> Tuple[float, float]:
    super_phi = np.arctan2(super_xyz[1], super_xyz[0])
    sub_phi   = np.arctan2(  sub_xyz[1],   sub_xyz[0])
    intersect = circle_line_intersection(super_xyz[:2], sub_xyz[:2], np.array([0, 0]), radius)
    if intersect is not None:
        bounds2 = np.arctan2(intersect[1], intersect[0])
        if phi_lies_within_bounds(0, (sub_phi, super_phi)):
            return (bounds2, sub_phi)
        else:
            return (sub_phi, bounds2)
    else:
        if phi_lies_within_bounds(0, (sub_phi, super_phi)):
            return (sub_phi - np.pi / 2, sub_phi)
        else:
            return (sub_phi, sub_phi + np.pi / 2)

In [ ]:
print(phi_bounds_from_lines(np.array((10, 10)), np.array((8, 5)), 8))

In [3]:
%%time
frame  = pd.read_csv("data/sets/RAMP-10N-25T-3600E-235R.gz")
frame  = utils.remove_padding(utils.remove_noise(frame))
frame  = frame.groupby(["event_id", "cluster_id"]).filter(lambda group: len(group) == 9)
events = utils.list_of_groups(frame, "event_id")
events = sorted(events, key=(lambda event: len(event)))

Wall time: 4.58 s


In [95]:
order     = ["phi", "r", "z"]
number    = 701
print("Hits:   {}".format(metrics.number_of_hits(events[number])))
print("Tracks: {}".format(metrics.number_of_tracks(events[number])))
positions = extractor.extract_input(events[number], order)
tracks    = extractor.extract_output(events[number], order, categorical=False)

Hits:   36
Tracks: 4


In [121]:
G = nx.DiGraph()
for idx, position in enumerate(positions):
    G.add_node(
        n=idx,
        cartesian=cartesian(position, order)[:2],
        phi=position[order.index("phi")],
        r=position[order.index("r")],
        z=position[order.index("z")]
    )
nodes    = G.nodes(data=True)
radiuses = groupby(positions[:, order.index("r")])
print(radiuses)

[[0, 16, 19, 35], [1, 17, 18, 34], [2, 15, 20, 33], [3, 14, 21, 32], [4, 13, 22, 31], [5, 12, 23, 30], [6, 11, 24, 29], [8, 9, 25, 28], [7, 10, 26, 27]]


In [122]:
def connect( 
        radiuses_index : int, 
        parents        : List[int],
        ) -> List[int]:
      # 1D array
    bounds     = (0, 2 * np.pi)
    candidates = radiuses[radiuses_index]  # List[int]
    radius     = nodes[candidates[0]][1]["r"]
    
    for parent_index in parents:
        cartesian = nodes[parent_index][1]["cartesian"]
        bounds    = tighten_bounds(bounds, phi_bounds_from_circle(cartesian, radius))
    if len(parents) > 1:
        c1 = nodes[parents[-1]][1]["cartesian"]
        c2 = nodes[parents[-2]][1]["cartesian"]
        bounds = tighten_bounds(bounds, phi_bounds_from_lines(c1, c2, radius))

    accepted = []
    for i in candidates:
        node = nodes[i][1]
        phi  = node["phi"]
        if phi_lies_within_bounds(phi, bounds):
            if radiuses_index <= 0:
                accepted.append((parents[-1], i))
            else:
                next_branches = connect(radiuses_index - 1, parents + [i])
                if next_branches:
                    accepted += next_branches + [(parents[-1], i)]
    return accepted

for i in radiuses[-1]:
    G.add_edges_from(connect(len(radiuses) - 2, [i]))
    break
G.add_node(n=-1, cartesian=np.array([0, 0]), phi=np.NaN, r=-1, z=0)

In [123]:
ax = plt.subplot(111)
for radius in np.unique(frame["r"]):
    ax.add_artist(plt.Circle((0, 0), radius, color="black", alpha=0.1, fill=False))
ax.set_xlim(-1100, 1100)
ax.set_ylim(-1100, 1100)
nx.draw_networkx(
    G=G,
    pos=nx.get_node_attributes(G, "cartesian"),
    ax=ax,
    node_size=200, 
    node_color="blue", 
    edge_color="purple", 
    alpha=.3,
    #style="dashed",
    font_color="black"
)

<IPython.core.display.Javascript object>